同数据相关分析：https://www.freesion.com/article/8705767400/


In [14]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import metrics

In [15]:
# import MedialCentre.xlsx as pandas Dataframe
df_raw = pd.read_excel("MedicalCentre.xlsx")
# rename wrong column name
df_raw.rename(columns = {'No-show':'No_show'},inplace = True)

In [16]:
# 去掉无用的feature
# df_raw.drop('index', axis=1, inplace=True)
df_raw.drop('PatientId', axis=1, inplace=True)
df_raw.drop('AppointmentID', axis=1, inplace=True)

#时间化数据
df_raw['ScheduledDay'] = pd.to_datetime(df_raw['ScheduledDay'], errors='ignore')
df_raw['AppointmentDay'] = pd.to_datetime(df_raw['AppointmentDay'], errors='ignore')

#计算时间间隔
df_raw['WaitingTime'] = df_raw['AppointmentDay'] - df_raw['ScheduledDay']

# WaitingTime列取天数,当天为-1，0
df_raw['WaitingTime']= df_raw['WaitingTime'].apply(lambda x: x.days)
 
# 过滤数据
df_raw = df_raw[df_raw['WaitingTime']>-1]
 
# df_raw.WaitingTime.describe()

In [17]:
# #时间标签
# # time_names = ['Sameday','OneDay','OneWeek','TwoWeek','ThreeWeek','OneMonth']
# time_names = [0,1,2,3,4,5]
# #把数据“分割”成组的边缘列表
# time_edges = [-2,0,1,7,13,18,179]

# # 创建 WaitingDays 列
# df_raw['WaitingDays'] = pd.cut(df_raw['WaitingTime'], time_edges, labels=time_names)

# # 显示数据框的前几行，确认更改
# df_raw.head()

In [18]:
# 对 waiting time 进行数据归一化
# df_raw['WaitingTime'] = (df_raw['WaitingTime'] - df_raw['WaitingTime'].mean()) / df_raw['WaitingTime'].std()
df_raw['WaitingTime'] = (df_raw['WaitingTime'] - df_raw['WaitingTime'].min()) / (df_raw['WaitingTime'].max() - df_raw['WaitingTime'].min())

In [19]:
# #清除年龄异常值
# df_raw = df_raw[df_raw['Age']>=0]

# #将'Age'列拆分成不同年龄段
# #时间标签
# age_names = [0, 1, 2, 3, 4]
# #把数据“分割”成组的边缘列表
# age_edges = [0,6,17,40,65,179]

# # 创建 Generation 列
# df_raw['Generation'] = pd.cut(df_raw['Age'], age_edges, labels=age_names)

# #标准化
# # df_raw['Generation'] = (df_raw['Generation']-df_raw['Generation'].min()) / (df_raw['Generation'].max() - df_raw['Generation'].min())
# # df_raw['Generation'].max()

# # 显示数据框的前几行，确认更改
# # df_raw.head()
# print(df_raw['Generation'])


In [20]:
#清除年龄异常值
df_raw = df_raw[df_raw['Age']>=0]
# 对年龄进行归一化
df_raw['Age'] = (df_raw['Age'] - df_raw['Age'].min()) / (df_raw['Age'].max() - df_raw['Age'].min())

In [21]:
# 对性别和No show进行二值化分类
df_raw["Gender"] = df_raw['Gender'].replace({'F': 0, 'M': 1})
df_raw["No_show"] = df_raw['No_show'].replace({'No': 0, 'Yes': 1})

In [22]:
# 对neighbourhood进行硬编码
neighbourmap = {elem:index+1 for index,elem in enumerate(set(df_raw["Neighbourhood"]))}
df_raw['Neighbourhood'] = df_raw['Neighbourhood'].map(neighbourmap)
# 归一化
df_raw['Neighbourhood'] = (df_raw['Neighbourhood'] - df_raw['Neighbourhood'].min()) / (df_raw['Neighbourhood'].max() - df_raw['Neighbourhood'].min())

In [23]:
df_raw.drop('ScheduledDay', axis=1, inplace=True)
df_raw.drop('AppointmentDay', axis=1, inplace=True)
temp = df_raw.pop('WaitingTime')
df_raw.insert(2, 'WaitingTime', temp)


In [24]:
y = df_raw.pop('No_show')
X = df_raw.copy()

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)

In [26]:
# naive_bayes = GaussianNB()
naive_bayes = MultinomialNB()
 
#Fitting the data to the classifier
naive_bayes.fit(X_train , y_train)
 
#Predict on test data
y_predicted = naive_bayes.predict(X_test)

metrics.accuracy_score(y_predicted , y_test)

0.5900500277932185